## 1. Подготовка к работе

In [2]:
# !pip install --upgrade tensorflow

In [3]:
# !pip install nltk
# !pip install tf-keras

In [4]:
import re
from collections import defaultdict
import nltk
import pandas as pd
from torch.utils.data import Dataset
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import Trainer, TrainingArguments 

In [5]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pront\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

##  2. Загрузка и предварительная обработка данных

### 2.1. Загрузка данных

In [18]:
df = pd.read_csv('data/work_data.csv')
df.head()

,address,name_ru,rating,rubrics,text
0,"Екатеринбург, ул. Московская / ул. Волгоградск...",Московский квартал,3,Жилой комплекс,Московский квартал 2. Шумно : летом по ночам д...
1,"Московская область, Электросталь, проспект Лен...",Продукты Ермолино,5,Магазин продуктов;Продукты глубокой заморозки;...,"Замечательная сеть магазинов в общем, хороший ..."
2,"Краснодар, Прикубанский внутригородской округ,...",LimeFit,1,Фитнес-клуб,"Не знаю смутят ли кого-то данные правила, но я..."
3,"Санкт-Петербург, проспект Энгельса, 111, корп. 1",Snow-Express,4,Пункт проката;Прокат велосипедов;Сапсёрфинг,Хорошие условия аренды. Дружелюбный персонал....
4,"Тверь, Волоколамский проспект, 39",Студия Beauty Brow,5,"Салон красоты;Визажисты, стилисты;Салон бровей...",Топ мастер Ангелина топ во всех смыслах ) Немн...


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 498918 entries, 0 to 498917
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   address  498918 non-null  object
 1   name_ru  498918 non-null  object
 2   rating   498918 non-null  int64 
 3   rubrics  498918 non-null  object
 4   text     498918 non-null  object
dtypes: int64(1), object(4)
memory usage: 19.0+ MB


### 2.2. Фильтрация и очистка данных

Возьмем данные, которые не содержат короткие отзывы (меньше 10 символов) и не содержат пропусков.

In [20]:
def clean_review_text(text: str) -> str:
    text = text.lower()  # Приводим к нижнему регистру
    text = re.sub(r"<[^>]+>", "", text)  # Удаляем HTML-теги
    text = re.sub(r"[^\w\s,.!?()]+", "", text)  # Удаляем спецсимволы, кроме пунктуации и скобок
    text = re.sub(r"\)\)+", " ", text)  # Заменяем смайлики вида "))" на пробел
    text = re.sub(r"\s+", " ", text).strip()  # Убираем лишние пробелы
    text = re.sub(r"[\n\r]+", " ", text)  # Заменяем переносы строк на пробелы
    return text


def clean_rubrics(rubrics_: str) -> str:
    return rubrics_.lower().strip()


def clean_name_ru(name_ru: str) -> str:
    return name_ru.lower().strip()


def clean_address(address: str) -> str:
    address = address.strip()
    return re.sub(r"\s+", " ", address)


df['text'] = df['text'].apply(clean_review_text)
df['rubrics'] = df['rubrics'].apply(clean_rubrics)
df['name_ru'] = df['name_ru'].apply(clean_name_ru)
df['address'] = df['address'].apply(clean_address)

# Преобразование рейтинга в числовой формат
df['rating'] = df['rating'].astype(float)

# Удаление строк с пропущенными значениями
df.dropna(subset=['address', 'name_ru', 'rubrics', 'rating', 'text'], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 498918 entries, 0 to 498917
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   address  498918 non-null  object 
 1   name_ru  498918 non-null  object 
 2   rating   498918 non-null  float64
 3   rubrics  498918 non-null  object 
 4   text     498918 non-null  object 
dtypes: float64(1), object(4)
memory usage: 19.0+ MB


In [21]:
df.head()

,address,name_ru,rating,rubrics,text
0,"Екатеринбург, ул. Московская / ул. Волгоградск...",московский квартал,3.0,жилой комплекс,московский квартал 2. шумно летом по ночам дик...
1,"Московская область, Электросталь, проспект Лен...",продукты ермолино,5.0,магазин продуктов;продукты глубокой заморозки;...,"замечательная сеть магазинов в общем, хороший ..."
2,"Краснодар, Прикубанский внутригородской округ,...",limefit,1.0,фитнес-клуб,"не знаю смутят ли когото данные правила, но я ..."
3,"Санкт-Петербург, проспект Энгельса, 111, корп. 1",snow-express,4.0,пункт проката;прокат велосипедов;сапсёрфинг,хорошие условия аренды. дружелюбный персонал. ...
4,"Тверь, Волоколамский проспект, 39",студия beauty brow,5.0,"салон красоты;визажисты, стилисты;салон бровей...",топ мастер ангелина топ во всех смыслах ) немн...


### 2.3. Создание входного запроса для модели

Мы будем формировать входной текст для модели, который будет содержать категорию, рейтинг и ключевые слова. В качестве ключевых слов будем использовать рубрики.

In [22]:
PROMT_PATTERN = """
Категория: {categories}
Рейтинг: {rating}
Ключевые слова: {keywords}
Отзыв:
"""

def preprocess_row(row):
    categories = row['rubrics'].split(';')  # Предполагается, что рубрики разделены запятыми
    categories = [cat.strip() for cat in categories]
    categories_str = ', '.join(categories)
    keywords = " ".join(categories)  # Можно дополнительно извлечь ключевые слова из текста
    formatted_text = PROMT_PATTERN.format(categories=categories_str, rating=row['rating'], keywords=keywords)
    return formatted_text

df['input_text'] = df.apply(preprocess_row, axis=1)
df['target_text'] = df['text'].str.strip()

df.head()

,address,name_ru,rating,rubrics,text,input_text,target_text
0,"Екатеринбург, ул. Московская / ул. Волгоградск...",московский квартал,3.0,жилой комплекс,московский квартал 2. шумно летом по ночам дик...,\nКатегория: жилой комплекс\nРейтинг: 3.0\nКлю...,московский квартал 2. шумно летом по ночам дик...
1,"Московская область, Электросталь, проспект Лен...",продукты ермолино,5.0,магазин продуктов;продукты глубокой заморозки;...,"замечательная сеть магазинов в общем, хороший ...","\nКатегория: магазин продуктов, продукты глубо...","замечательная сеть магазинов в общем, хороший ..."
2,"Краснодар, Прикубанский внутригородской округ,...",limefit,1.0,фитнес-клуб,"не знаю смутят ли когото данные правила, но я ...",\nКатегория: фитнес-клуб\nРейтинг: 1.0\nКлючев...,"не знаю смутят ли когото данные правила, но я ..."
3,"Санкт-Петербург, проспект Энгельса, 111, корп. 1",snow-express,4.0,пункт проката;прокат велосипедов;сапсёрфинг,хорошие условия аренды. дружелюбный персонал. ...,"\nКатегория: пункт проката, прокат велосипедов...",хорошие условия аренды. дружелюбный персонал. ...
4,"Тверь, Волоколамский проспект, 39",студия beauty brow,5.0,"салон красоты;визажисты, стилисты;салон бровей...",топ мастер ангелина топ во всех смыслах ) немн...,"\nКатегория: салон красоты, визажисты, стилист...",топ мастер ангелина топ во всех смыслах ) немн...


### 2.5. Разделение данных на обучающую и тестовую выборки


In [23]:
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

print(f"Размер обучающей выборки: {len(train_df)}")
print(f"Размер валидационной выборки: {len(val_df)}")

Размер обучающей выборки: 449026
Размер валидационной выборки: 49892


## 3. Создание датасета и загрузчика данных

Используем PyTorch для создания пользовательского датасета.

In [24]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer

class ReviewsDataset(Dataset):
    def __init__(self, inputs, targets, tokenizer, max_length=512):
        self.inputs = inputs
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        input_enc = self.tokenizer.encode_plus(
            self.inputs[idx],
            max_length=self.max_length,
            truncation=True,
            padding='max_length',
            return_tensors='pt'
        )
        target_enc = self.tokenizer.encode_plus(
            self.targets[idx],
            max_length=self.max_length,
            truncation=True,
            padding='max_length',
            return_tensors='pt'
        )
        labels = target_enc['input_ids']
        labels[labels == self.tokenizer.pad_token_id] = -100  # Игнорирование паддинга при расчете лосса

        return {
            'input_ids': input_enc['input_ids'].squeeze(),
            'attention_mask': input_enc['attention_mask'].squeeze(),
            'labels': labels.squeeze()
        }

# Выбор модели и токенизатора
model_name = 'sberbank-ai/rugpt3small_based_on_gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Добавление специальных токенов, если необходимо
# tokenizer.add_special_tokens({'pad_token': '[PAD]'})  # Например, добавление PAD токена

# Создание датасетов
train_dataset = ReviewsDataset(
    inputs=train_df['input_text'].tolist(),
    targets=train_df['target_text'].tolist(),
    tokenizer=tokenizer
)

val_dataset = ReviewsDataset(
    inputs=val_df['input_text'].tolist(),
    targets=val_df['target_text'].tolist(),
    tokenizer=tokenizer
)

# Создание загрузчиков данных
batch_size = 8

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)



## 4. Выбор и настройка модели

Мы будем использовать предобученную модель ruGPT-3 Small, предоставленную SberAI. Вы можете выбрать другую модель из Hugging Face Model Hub (https://huggingface.co/models).

In [25]:
# Загрузка предобученной модели
model = GPT2LMHeadModel.from_pretrained(model_name)

## 5. Обучение модели

Используем Trainer из Hugging Face для упрощения процесса обучения.

In [26]:
from transformers import AdamW, get_linear_schedule_with_warmup
import torch.nn as nn

# Проверка наличия GPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Параметры обучения
epochs = 3
learning_rate = 5e-5
warmup_steps = 500

# Определение оптимизатора и планировщика
optimizer = AdamW(model.parameters(), lr=learning_rate)

total_steps = len(train_loader) * epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps
)

# Критерий потерь
criterion = nn.CrossEntropyLoss(ignore_index=-100)

# Функция обучения на одной эпохе
def train_epoch(model, dataloader, optimizer, scheduler, device):
    model.train()
    total_loss = 0
    for batch in tqdm(dataloader, desc="Training"):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        scheduler.step()

    avg_loss = total_loss / len(dataloader)
    return avg_loss

# Функция валидации на одной эпохе
def eval_epoch(model, dataloader, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Validation"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    return avg_loss

from tqdm import tqdm

# Цикл обучения
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")

    train_loss = train_epoch(model, train_loader, optimizer, scheduler, device)
    print(f"Training loss: {train_loss:.4f}")

    val_loss = eval_epoch(model, val_loader, device)

    print(f"Validation loss: {val_loss:.4f}")

    # Можно добавить сохранение модели после каждой эпохи
    model.save_pretrained(f'model_epoch_{epoch + 1}')
    tokenizer.save_pretrained(f'model_epoch_{epoch + 1}')


# После завершения обучения сохраним модель и токенизатор для дальнейшего использования.
model_save_path = 'models/model_v2'
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)


c:\Users\pront\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(



Epoch 1/3


Training: 100%|██████████| 56129/56129 [3:27:33<00:00,  4.51it/s]  


Training loss: 7.2036


Validation: 100%|██████████| 6237/6237 [07:28<00:00, 13.91it/s]


...

Training: 100%|██████████| 56129/56129 [3:31:42<00:00,  4.42it/s]  


Training loss: 7.1056


Validation: 100%|██████████| 6237/6237 [07:27<00:00, 13.94it/s]


Validation loss: 7.1199


('models/model_v2\\tokenizer_config.json',
 'models/model_v2\\special_tokens_map.json',
 'models/model_v2\\vocab.json',
 'models/model_v2\\merges.txt',
 'models/model_v2\\added_tokens.json')

In [27]:
def generate_review(category, rating, keywords, max_length=150, temperature=0.7, top_k=50, top_p=0.95):
    prompt = f"Категория: {category}\nРейтинг: {rating}\nКлючевые слова: {keywords}\nОтзыв:"
    inputs = tokenizer.encode(prompt, return_tensors='pt').to(device)

    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_length=inputs.shape[1] + max_length,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            do_sample=True,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id
        )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Извлечение только текста отзыва
    review = generated_text.split('Отзыв:')[-1].strip()
    return review


category = "ресторан"
rating = 5
keywords = "паста"

generated_review = generate_review(category, rating, keywords, 15, 1, 100000, 1.5)
print("Сгенерированный отзыв:")
print(generated_review)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Сгенерированный отзыв:
и них великная и ожидание куб вкус своей есть оченьняя понравилась своему прив


In [ ]:
model.save_pretrained("models/model_v1")
tokenizer.save_pretrained("models/model_v1")